In [14]:
import pandas as pd 
from faker import Faker
import random
from datetime import datetime, timedelta, date
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from dotenv import load_dotenv
from pathlib import Path
import os

In [4]:
fake = Faker()
fake_unique = fake.unique

In [5]:
def random_date(start_days_ago=365, end_days_ago=1):
    start_date = datetime.now() - timedelta(days=start_days_ago)
    end_date = datetime.now() - timedelta(days=end_days_ago)
    return fake.date_time_between(start_date=start_date, end_date=end_date)

In [6]:
NUM_CUSTOMERS = 5001
NUM_SERVICE_AREA = 101
NUM_EMPLOYEES = 1001
NUM_WAREHOUSES = 51
NUM_VENDORS = 201
NUM_VEHICLES = 501
NUM_MAINTENANCE = 3001
NUM_FUEL_PURCHASES = 5001
NUM_SHIPMENTS = 50001
NUM_ROUTES = 3001
NUM_CLAIMS = 1001
NUM_TICKETS = 1501
NUM_PACKAGES = 100001
NUM_ROUTE_STOP = 30000
NUM_INVENTORY = 100000
NUM_INVOICES = 50001

In [52]:
customers = []

for i in range(NUM_CUSTOMERS):
    customers.append({
        'customer_id': i,
        'name': fake.name(),
        'email': fake_unique.email(),
        'phone': fake_unique.phone_number(),
        'address_line1': fake.street_address(),
        'address_line2': fake.street_address(),
        'city': fake.city(),
        'state': fake.state(),
        'postal_code': fake.postalcode(),
        'created_at': fake.date_time(),
        'customer_type': fake.random_element(['individual', 'business']),
        'preferred_contact_method': fake.random_element(['email', 'sms']),
        'loyalty_points': fake.random_int(min=1, max=1000)
    })
customers_df = pd.DataFrame(customers)

In [22]:
service_areas = []

for i in range(NUM_SERVICE_AREA):
    service_areas.append({
        'area_id': i, 
        'name': f"{fake.city()} Area",
        'description': fake.text(max_nb_chars=50),
        'base_delivery_fee': round(random.uniform(5, 50), 2),
        'estimated_transit_time_hours': random.randint(1, 48),
        'service_level': random.choice(['standard', 'expedited', 'same-day'])
    })
df_service_areas = pd.DataFrame(service_areas)

In [23]:
zip_codes = []
used_zips = set()
max_retries = 100

service_areas_list = (
    service_areas.to_dict('records') 
    if isinstance(service_areas, pd.DataFrame) 
    else service_areas
)

for area in service_areas_list:
    zips_for_area = set()  
    num_zips = random.randint(5, 15)  
    
    while len(zips_for_area) < num_zips:
        retries = 0
        while retries < max_retries:
            z = fake.postcode()
            if z not in used_zips and z not in zips_for_area:
                break
            retries += 1
        
        if retries == max_retries:
            raise ValueError(f"Failed to generate unique zip code for area {area['area_id']}")
        
        zip_codes.append({
            'area_id': area['area_id'],
            'zip_code': z
        })
        used_zips.add(z)
        zips_for_area.add(z)

df_zip_codes = pd.DataFrame(zip_codes)

In [54]:
employees = []

for i in range(NUM_EMPLOYEES):
    possible_manager_ids = [e['employee_id'] for e in employees] if i > 10 and random.random() < 0.7 else []
    manager_id = random.choice(possible_manager_ids) if possible_manager_ids else None

    employees.append({
        'employee_id': i,
        'first_name': fake.first_name(),
        'last_name': fake.last_name(),
        'email': fake.unique.email(),
        'phone': fake.unique.phone_number(),
        'hire_date': fake.date_between(start_date='-5y', end_date='today'),
        'job_title': random.choice(['Driver', 'Manager', 'Dispatcher', 'Clerk', 'Technician']),
        'department': random.choice(['Operations', 'HR', 'IT', 'Logistics']),
        'manager_id': manager_id,
        'monthly_salary': round(random.uniform(2000, 8000), 2),
        'address': fake.address(),
        'emergency_contact': fake.name(),
        'driver_license_number': fake.unique.bothify(text='??######')
    })

df_employees = pd.DataFrame(employees)


In [25]:
warehouses = []

for i in range(NUM_WAREHOUSES):
    warehouses.append({
        'warehouse_id': i,
        'name': f"Warehouse {i}",
        'address': fake.unique.address(),
        'city': fake.city(),
        'state': fake.state(),
        'postal_code': fake.postcode(),
        'created_at': fake.date_between(start_date='-10y', end_date='-1y'),
        'updated_at': fake.date_between(start_date='-1y', end_date='today'),
        'square_footage': random.randint(1000, 10000),
        'dock_doors_count': random.randint(2, 20),
        'start_time': fake.time_object(end_datetime=None),
        'end_time': fake.time_object(end_datetime=None),
        'manager_id': random.randint(1, 1000),
        'temperature_controlled': fake.boolean(),
        'hazardous_material_allowed': fake.boolean()
    })
df_warehouses = pd.DataFrame(warehouses)

In [26]:
vendors = []

for i in range(NUM_VENDORS):
    vendors.append({
        'vendor_id': i,
        'name': fake.company(),
        'contact_person': fake.name(),
        'email': fake.company_email(),
        'phone': fake.phone_number(),
        'service_type': random.choice(['fuel', 'maintenance', 'insurance']),
        'contract_start_date': fake.date_between(start_date='-5y', end_date='-1y'),
        'contract_end_date': fake.date_between(start_date='today', end_date='+2y'),
        'preferred_vendor_status': random.choice(['preferred', 'non-preferred'])
    })
df_vendors = pd.DataFrame(vendors)

In [ ]:
vehicles = []

for i in range(NUM_VEHICLES):  
    year = random.randint(2010, 2024)
    acquisition = fake.date_between(start_date=date(year, 1, 1), end_date=date(year, 12, 31))
    last_insp = fake.date_between(start_date=acquisition, end_date='today')
    next_insp = fake.date_between(start_date='today', end_date='+1y')

    vehicles.append({
        'vehicle_id': i,
        'license_plate': fake_unique.bothify(text='??###??'),
        'make': fake.company(),
        'model': fake.word(),
        'year': year,
        'vehicle_type': random.choice(['van', 'box-truck']),
        'max_weight_capacity': random.randint(1000, 10000),
        'max_volume_capacity': random.randint(200, 1500),
        'current_status': random.choice(['available', 'in-maintenance', 'out-of-service']),
        'acquisition_date': acquisition,
        'last_inspection_date': last_insp,
        'next_inspection_date': next_insp,
        'fuel_type': random.choice(['gasoline', 'diesel', 'electric', 'hybrid']),
        'average_mpg': random.randint(5, 25),
        'current_mileage': random.randint(10000, 300000)
    })
df_vehicles = pd.DataFrame(vehicles)

In [28]:
maintenance_records = []

vehicles_list = vehicles.to_dict('records') if isinstance(vehicles, pd.DataFrame) else vehicles

for i in range(NUM_MAINTENANCE):
    vehicle = random.choice(vehicles_list)  # FIXED: Works for both DataFrames and lists
    mileage = random.randint(5000, vehicle['current_mileage'])
    maintenance_date = fake.date_between(start_date=vehicle['acquisition_date'], end_date='today')
    
    maintenance_records.append({
        'maintenance_id': i,
        'vehicle_id': vehicle['vehicle_id'],  # Correct reference
        'vendor_id': random.randint(0, NUM_VENDORS - 1),  # FIXED: 0-199 (for NUM_VENDORS=200)
        'maintenance_type': random.choice(['oil change', 'brake inspection', 'tire rotation', 'engine check']),
        'description': fake.sentence(nb_words=6),
        'date_performed': maintenance_date,
        'mileage_at_service': mileage,
        'cost': round(random.uniform(50, 1000), 2),
        'part_used': fake.word(),
        'next_service_date': fake.date_between(start_date=maintenance_date, end_date='+6m'),
        'technician_note': fake.sentence(nb_words=8)
    })

df_maintenance = pd.DataFrame(maintenance_records)

In [29]:
fuel_purchases = []

for i in range(NUM_FUEL_PURCHASES):  
    gallons = round(random.uniform(5, 50), 2)
    price = round(random.uniform(2.5, 6.0), 2)
    total = round(gallons * price, 2)
    purchase_dt = fake.date_time_between(start_date='-2y', end_date='now')

    fuel_purchases.append({
        'purchase_id': i,
        'vehicle_id': random.randint(1, NUM_VEHICLES -1),
        'vendor_id': random.randint(1, NUM_VENDORS -1),
        'purchase_date': purchase_dt,
        'gallons': gallons,
        'price_per_gallon': price,
        'total_cost': total,
        'odometer_reading': round(random.uniform(5000, 300000), 2),
        'payment_method': random.choice(['cash', 'credit-cards', 'debit-cards', 'fuel-cards']),
        'receipt_number': fake.unique.bothify(text='RCPT-####-???'),
        'driver_id': random.randint(1, 500)
    })
df_fuel = pd.DataFrame(fuel_purchases)

In [63]:
shipments = []

for i in range(NUM_SHIPMENTS): 
    pickup = fake.date_between(start_date='-2y', end_date='-1d')
    est_delivery = pickup + timedelta(days=random.randint(1, 5))
    actual_delivery = est_delivery + timedelta(days=random.choice([-1, 0, 1, 2]))

    shipments.append({
        'shipment_id': i,
        'customer_id': random.randint(1, 1000),
        'origin_warehouse_id': random.randint(0, NUM_WAREHOUSES -1),
        'destination_address': fake.street_address(),
        'destination_city': fake.city(),
        'destination_state': fake.state(),
        'destination_postal_code': fake.zipcode(),
        'service_area_id': random.randint(1, 100),
        'pickup_date': pickup,
        'estimated_delivery_date': est_delivery,
        'actual_delivery_date': actual_delivery,
        'status': random.choice(['pending', 'in-transit', 'delivered', 'delayed', 'cancelled']),
        'priority_level': random.choice(['low', 'medium', 'high', 'urgent']),
        'special_instructions': fake.sentence(nb_words=6),
        'dispatcher_id': random.randint(1, 500),
        'billing_notes': fake.sentence(nb_words=5)
    })
df_shipments = pd.DataFrame(shipments)

In [31]:
routes = []

driver_ids = [
    emp['employee_id'] for emp in employees 
    if emp['job_title'] == 'Driver'
]

for i in range(NUM_ROUTES): 
    start_at = fake.date_time_between(start_date='-2y', end_date='-1d')
    end_at = start_at + timedelta(hours=random.randint(2, 10))

    routes.append({
        'route_id': i,
        'driver_id': random.choice(driver_ids),  # FIXED: Only valid drivers
        'vehicle_id': random.randint(0, NUM_VEHICLES - 1),  # FIXED: 0-499
        'start_warehouse_id': random.randint(0, NUM_WAREHOUSES - 1),  # FIXED: 0-49
        'start_at': start_at,
        'end_at': end_at,
        'status': random.choice(['scheduled', 'in-progress', 'paused', 'completed', 'cancelled']),
        'total_stops_planned': random.randint(3, 20),
        'total_stops_completed': random.randint(0, 20),  # Can be 0 if cancelled
        'miles_estimated': round(random.uniform(10, 300), 2),
        'miles_actual': round(random.uniform(10, 350), 2),
        'fuel_consumed_gallons': round(random.uniform(2, 40), 2),
        'route_score_by_driver': round(random.uniform(1, 5), 2)
    })

df_routes = pd.DataFrame(routes)

In [32]:
insurance_claims = []

for i in range(NUM_CLAIMS): 
    claim_date = fake.date_between(start_date='-1y', end_date='-1d')
    status = random.choice(['pending', 'approved', 'denied'])
    settlement_date = claim_date + timedelta(days=random.randint(5, 30)) if status != 'pending' else None
    settlement_amount = round(random.uniform(50, 500), 2) if settlement_date else None

    insurance_claims.append({
        'claim_id': i,
        'shipment_id': random.randint(1, 5000),
        'customer_id': random.randint(1, 1000),
        'claim_date': claim_date,
        'claim_amount': round(random.uniform(50, 1000), 2),
        'damage_description': fake.sentence(nb_words=5),
        'claim_status': status,
        'settlement_date': settlement_date,
        'settlement_amount': settlement_amount,
        'adjuster_id': random.randint(1, 500),
        'investigation_notes': fake.sentence(nb_words=6)
    })
df_claims = pd.DataFrame(insurance_claims)

In [69]:
tickets = []

for i in range(NUM_TICKETS):  # 1,500 tickets
    rating = round(random.uniform(1, 5), 1)
    ticket_date = fake.date_between(start_date='-1y', end_date='-1d')

    tickets.append({
        'ticket_id': i,
        'customer_id': random.randint(0, NUM_CUSTOMERS-1),
        'shipment_id': random.randint(0, NUM_SHIPMENTS-1),
        'employee_id': random.randint(0, NUM_EMPLOYEES-1),
        'ticket_date': ticket_date,
        'issue_type': random.choice(['delivery_delay', 'damaged_package', 'missing_item', 'billing_issue']),
        'description': fake.sentence(nb_words=6),
        'resolution': random.choice(['Refund issued', 'Resent package', 'Escalated to billing', 'Investigation closed']),
        'status': random.choice(['open', 'in-progress', 'resolved', 'escalated']),
        'priority': random.choice(['low', 'medium', 'high', 'urgent']),
        'resolution_time_minutes': random.randint(10, 1000),
        'customer_satisfaction_rating': rating
    })
df_tickets = pd.DataFrame(tickets)

In [ ]:
packages = []

for i in range(NUM_PACKAGES):
    packages.append({
        'package_id': i,
        'shipment_id': random.randint(0, NUM_SHIPMENTS-1),
        'description': fake.sentence(nb_words=4),
        'weight': round(random.uniform(0.5, 75), 2),
        'height': round(random.uniform(5, 100), 2),
        'width': round(random.uniform(5, 100), 2),
        'length': round(random.uniform(5, 100), 2),
        'package_type': random.choice(['Box', 'Envelope', 'Crate', 'Tube']),
        'tracking_number': fake_unique.bothify(text='TRK#######??'),
        'is_fragile': random.choice([True, False]),
        'is_perishable': random.choice([True, False]),
        'requires_signature': random.choice([True, False]),
        'declared_value': round(random.uniform(10, 2000), 2),
        'customs_info': fake.sentence(nb_words=3),
        'handling_instructions': fake.sentence(nb_words=5)
    })
df_packages = pd.DataFrame(packages)

In [67]:
route_stops = []

for i in range(NUM_ROUTE_STOP):
    estimated_arrival = fake.date_time_this_year()
    actual_arrival = estimated_arrival + timedelta(minutes=random.randint(0, 60))
    departure_time = actual_arrival + timedelta(minutes=random.randint(5, 30))

    route_stops.append({
        'stop_id': i,
        'route_id': random.randint(0, NUM_ROUTES-1),
        'sequence_number': random.randint(1, 10),
        'shipment_id': random.randint(0, NUM_SHIPMENTS-1),
        'estimated_arrival': estimated_arrival,
        'actual_arrival': actual_arrival,
        'departure_time': departure_time,
        'stop_duration_minutes': random.randint(5, 30),
        'stop_status': random.choice(['pending', 'completed', 'skipped', 'failed']),
        'delivery_notes': fake.sentence(nb_words=4),
        'recipient_signature': fake.name(),
        'proof_of_delivery_url': fake.image_url()
    })
df_route_stops = pd.DataFrame(route_stops)

In [86]:
inventory = []

for i in range(NUM_INVENTORY):
    date_received = fake.date_time_this_year()
    date_shipped = date_received + timedelta(days=random.randint(1, 5)) if random.random() > 0.3 else None

    inventory.append({
        'inventory_id': i,
        'warehouse_id': random.randint(0, NUM_WAREHOUSES -1),
        'package_id': random.randint(0, NUM_PACKAGES -1),
        'location_code': f"LOC-{fake_unique.lexify(text='??????????')}",
        'date_received': date_received,
        'date_shipped': date_shipped,
        'current_status': random.choice(['received', 'staged', 'loaded', 'delivered']),
        'handled_by': random.randint(0, NUM_EMPLOYEES -1) if random.random() > 0.1 else None,
        'temperature_log': round(random.uniform(-5.0, 35.0), 2)
    })
df_inventory = pd.DataFrame(inventory)

In [71]:
invoices = []

for i in range(NUM_INVOICES):
    issue_date = fake.date_time_this_year()
    due_date = issue_date + timedelta(days=15)
    total_amount = round(random.uniform(50, 500), 2)
    tax_amount = round(total_amount * 0.1, 2)
    discount_amount = round(random.uniform(0, 20), 2)
    amount_paid = total_amount - discount_amount if random.random() > 0.1 else 0
    payment_date = issue_date + timedelta(days=random.randint(1, 20)) if amount_paid > 0 else None
    payment_status = 'paid' if amount_paid >= total_amount - discount_amount else random.choice(['pending', 'overdue', 'refunded'])

    invoices.append({
        'invoice_id': i,
        'shipment_id': random.randint(0, NUM_SHIPMENTS-1),
        'customer_id': random.randint(0, NUM_CUSTOMERS-1),
        'issue_date': issue_date,
        'due_date': due_date,
        'total_amount': total_amount,
        'tax_amount': tax_amount,
        'discount_amount': discount_amount,
        'amount_paid': amount_paid,
        'payment_date': payment_date,
        'payment_method': random.choice(['credit_card', 'bank_transfer', 'paypal', 'cash']),
        'payment_status': payment_status,
        'late_fee_amount': round(random.uniform(0, 10), 2) if payment_status == 'overdue' else 0.0
    })
df_invoices = pd.DataFrame(invoices)

In [38]:
env_path = Path('.') / 'config.env'
load_dotenv(dotenv_path=env_path)

True

In [39]:
tables = {
    'customers_df': 'customers',
    'df_service_areas': 'service_areas',
    'df_zip_codes': 'service_area_zip_codes',
    'df_employees': 'employees',
    'df_warehouses': 'warehouses',
    'df_vendors': 'vendors',
    'df_vehicles': 'vehicles',
    'df_maintenance': 'maintenance_records',
    'df_fuel': 'fuel_purchases',
    'df_shipments': 'shipments',
    'df_routes': 'delivery_routes',
    'df_claims': 'insurance_claims',
    'df_tickets': 'tickets',
    'df_packages': 'packages',
    'df_route_stops': 'route_stops',
    'df_inventory': 'inventory',
    'df_invoices': 'invoices'
}

In [ ]:
engine = create_engine(
    f"mysql+pymysql://{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}@"
    f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}",
    pool_size=10,              
    max_overflow=20,
    pool_pre_ping=True         
)

# Test connection
try:
    with engine.connect() as conn:
        print("Database connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")

Connection failed: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'None' ([Errno 11001] getaddrinfo failed)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [87]:
try:
    with engine.begin() as connection:  # Starts a transaction
        for df_var, table_name in tables.items():
            df = globals()[df_var]
            print(f"Appending to {table_name}...")

            df.to_sql(name=table_name, con=connection, if_exists='append', index=False)
            print(f"✅ Data appended to {table_name} successfully.")

except SQLAlchemyError as e:
    print(f"❌ Transaction failed. Rolled back all changes. Error: {e}")


Appending to customers...
✅ Data appended to customers successfully.
Appending to service_areas...
✅ Data appended to service_areas successfully.
Appending to service_area_zip_codes...
✅ Data appended to service_area_zip_codes successfully.
Appending to employees...
✅ Data appended to employees successfully.
Appending to warehouses...
✅ Data appended to warehouses successfully.
Appending to vendors...
✅ Data appended to vendors successfully.
Appending to vehicles...
✅ Data appended to vehicles successfully.
Appending to maintenance_records...
✅ Data appended to maintenance_records successfully.
Appending to fuel_purchases...
✅ Data appended to fuel_purchases successfully.
Appending to shipments...
✅ Data appended to shipments successfully.
Appending to delivery_routes...
✅ Data appended to delivery_routes successfully.
Appending to insurance_claims...
✅ Data appended to insurance_claims successfully.
Appending to tickets...
✅ Data appended to tickets successfully.
Appending to packages